## [Dacon] AI프렌즈 시즌2 강수량 산출 경진대회
## 팀: endgame
## 2020년 월 일 (제출날짜)

## <div style="color:red">README</div>
- 하드웨어 리소스가 많이 소요되는 코드입니다. 제 컴퓨터의 램이 128GB라서 모든 데이터를 램에 올려놓고 작업을 실시했습니다.
- 저 같은 경우, EDA, 모델링 각각 다른 ipynb 파일에서 작업을 진행했는데, 제출용 파일이다보니 모든 코드를 한 곳에 모아 실행하기에 메모리가 부족할 가능성이 커질 것 같습니다.
- train.zip, test.zip 파일은 각각 data/train, data/test 폴더에 압축을 해제해주세요.
- sample_submission.csv는 data 폴더에 위치시켜 주세요.

<div style="color:red">혹시 위의 글을 안 읽으셨다면 꼭 읽어주세요!</div>

## 1. 라이브러리 및 데이터 (Library & Data)

In [ ]:
# 파일관리 및 파일선택
import os
import pickle
import random


# 시각화
import seaborn as sns
import matplotlib.pyplot as plt


import numpy as np
import pandas as pd
import gc
import tensorflow as tf
from custom_metric import *

SEED = 30
np.random.seed(SEED)
random.seed(SEED)
tf.random.set_seed(SEED)

## 2. 데이터 전처리 (Data Cleansing & Pre-Processing)

### 2.1 데이터셋을 만들고 pickle로 저장
- -9999와 같은 missing value가 들어있으면 제거
- 0.1 이상 내린 픽셀이 UPPER 값 이상인 사진만 데이터셋에 저장

In [ ]:
dir_train = 'data/train/'
dir_test = 'data/test/'
UPPER = 50

def make_dataset(dir_train, dir_test, UPPER):
    # train dataset
    train = []
    train_y = []

    for i in os.listdir(dir_train):
        npy = np.load(dir_train + i)

        # missing value 제거
        if npy[:, :, -1].sum() < 0:
            continue
        
        # 0.1이상 내린 픽셀이 UPPER 값 이상인 사진만
        if (npy[:, :, -1] >= 0.1).sum() >= UPPER:
            train.append(npy[:, :, :-1])
            train_y.append(npy[:, :, -1])

    train = np.array(train)
    train_y = np.array(train_y)

    with open(f'data/train{UPPER}.pickle', 'wb') as f:
        pickle.dump(train, f, protocol=4)

    with open(f'data/train_y{UPPER}.pickle', 'wb') as f:
        pickle.dump(train_y, f, protocol=4)

    del train
    del train_y

    # test dataset
    test = []

    for i in os.listdir(dir_test):
        npy = np.load(dir_test + i)
        test.append(npy)
    test = np.array(test)

    with open('data/test.pickle', 'wb') as f:
        pickle.dump(test, f, protocol=4)
    del test
    
make_dataset(dir_train, dir_test, 50)

### 2.2 pickle 파일 로드

In [ ]:
with open('data/train50.pickle', 'rb') as f:
    train = pickle.load(f)
    
# 0~9번채널만 사용
train = train[:, :, :, :10]

In [ ]:
with open('data/train_y50.pickle', 'rb') as f:
    train_y = pickle.load(f)
train_y = train_y.reshape(train_y.shape[0], 40, 40, 1)

train_y = np.log(train_y+1)    

In [ ]:
with open('data/test.pickle', 'rb') as f:
    TEST = pickle.load(f)
TEST = TEST[:, :, :, :10] 

## 3. 탐색적 자료분석 (Exploratory Data Analysis)

### 3.1 시각화를 이용한 EDA
- v별, h별 합계 피쳐를 만들고 강수량과의 관계를 시각적으로 파악해봤습니다.

In [ ]:
image_dir = os.listdir('inputs/train/')
image_sample = np.load(f'inputs/train/{image_dir[random.randrange(len(image_dir))]}')

def showimg(img):
    ch15_v = 0
    for i in [0,2,4,5,7]:
        ch15_v += img[:,:,i]
    ch15_h = 0
    for i in [1,3,6,8]:
        ch15_h += img[:,:,i]
    ch15_v = ch15_v.reshape(40,40,1)
    ch15_h = ch15_h.reshape(40,40,1)
    img = np.concatenate([img, ch15_v], -1)
    img = np.concatenate([img, ch15_h], -1)
    return img

image_sample = showimg(image_sample)

color_map = plt.cm.get_cmap('RdBu')
color_map = color_map.reversed()
plt.style.use('fivethirtyeight')
plt.figure(figsize=(10, 10))

for i in range(9):
    plt.subplot(2,6,i+1)
    plt.imshow(image_sample[:, :, i], cmap=color_map)
    plt.title(f'ch_{i}', fontdict= {'fontsize': 16})

plt.subplot(2,6,10)
plt.imshow(image_sample[:,:,-3], cmap = color_map)
plt.title('rain', fontdict= {'fontsize': 16})

plt.subplot(2,6,11)
plt.imshow(image_sample[:,:,-2], cmap = color_map)
plt.title('v_sum', fontdict= {'fontsize': 16})

plt.subplot(2,6,12)
plt.imshow(image_sample[:,:,-1], cmap = color_map)
plt.title('h_sum', fontdict= {'fontsize': 16})

plt.subplots_adjust(top=0.5)
plt.show()

### 3.2 상관관계를 이용한 EDA

## 4. 변수 선택 및 모델 구축 (Feature Engineering & Initial Modeling)

### 4.1 vertical, horizontal 별로 Sum한 피쳐 추가

In [ ]:
def channel_sum(data):
    data_v = data[:, :, :, 0].copy() + data[:, :, :, 2].copy() + data[:, :, :, 4].copy() + data[:, :, :, 5].copy() +data[:, :, :, 7].copy()
    data_h = data[:, :, :, 1].copy() + data[:, :, :, 3].copy() + data[:, :, :, 6].copy() + data[:, :, :, 8].copy()

    data_v = data_v.reshape(data_v.shape[0], data_v.shape[1], data_v.shape[2], 1)
    data_h = data_h.reshape(data_h.shape[0], data_h.shape[1], data_h.shape[2], 1)

    data = np.concatenate([data, data_v.copy()], -1)
    data = np.concatenate([data, data_h.copy()], -1)

    return data

train = channel_sum(train)
TEST = channel_sum(TEST)

### 4.2 9번 채널 정규화

In [ ]:
train[:, :, :, 9] = train[:, :, :, 9] / 322
TEST[:, :, :, 9] = TEST[:, :, :, 9] / 322
VAL_X[:, :, :, 9] = VAL_X[:, :, :, 9] / 322

## 5. 모델 학습 및 검증 (Model Tuning & Evaluation)

In [ ]:
# 입력하세요.

## 6. 결과 및 결언 (Conclusion & Discussion)

In [ ]:
# 입력하세요.